In [3]:
import numpy as np
!python -V

Python 3.9.12


In [4]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [5]:
# Read parquet for yellow trips 2023 
df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')

# df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
# df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

# df = df[(df.duration >= 1) & (df.duration <= 60)]

# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

# df[categorical] = df[categorical].astype(str)

In [6]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


## Questions 1: Read the data for January. How many columns are there?

In [7]:
# Get the number of columns
num_columns = df.shape[1]

print(f"Number of columns: {num_columns}")

Number of columns: 19


## Question 2: Now let's compute the duration variable. It should contain the duration of a ride in minutes. What's the standard deviation of the trips duration in January?

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
# df.duration_mins = df.duration_mins.apply(lambda td: td.total_seconds() / 60)
df['duration_minutes'] = df['duration'].dt.total_seconds() / 60

In [9]:
np.std(df['duration_minutes'])

42.5943442974141

## Question 3: Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive). What fraction of the records left after you dropped the outliers?

In [10]:
filtered_df = df[(df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)]

In [11]:
original_count = len(df)
filtered_df = df[(df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)]
filtered_count = len(filtered_df)
fraction_remaining = filtered_count / original_count

print(f"Fraction of records remaining: {fraction_remaining:.4f}")


Fraction of records remaining: 0.9812


## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [12]:
# Step 1: Convert location IDs to strings
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Step 2: Convert to list of dictionaries for one-hot encoding
dicts = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Step 3: Fit DictVectorizer and transform
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Step 4: Report dimensionality
print(f"Number of features: {X.shape[1]}")

# Step 5: Define target variable
y = filtered_df['duration_minutes'].values

# Step 6: Train linear regression model
model = LinearRegression()
model.fit(X, y)

# Step 7: Predict and calculate RMSE
y_pred = model.predict(X)
rmse = mean_squared_error(y, y_pred, squared=False)

print(f"RMSE on filtered training data: {rmse:.4f}")

/tmp/ipykernel_36728/4207838692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
/tmp/ipykernel_36728/4207838692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


Number of features: 515
RMSE on filtered training data: 7.6493


# Question 6: Validatoin on Feb Data 

In [13]:
df = pd.read_parquet('./yellow_tripdata_2023-02.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
# df.duration_mins = df.duration_mins.apply(lambda td: td.total_seconds() / 60)
df['duration_minutes'] = df['duration'].dt.total_seconds() / 60
filtered_df = df[(df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)]


# Step 1: Convert location IDs to strings
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Step 2: Convert to list of dictionaries for one-hot encoding
dicts = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Step 3: Fit DictVectorizer and transform
# dv = DictVectorizer()
X = dv.transform(dicts)

# Step 4: Report dimensionality
print(f"Number of features: {X.shape[1]}")

# Step 5: Define target variable
y = filtered_df['duration_minutes'].values

# Step 6: Train linear regression model
# model = LinearRegression()
# model.fit(X, y)

# Step 7: Predict and calculate RMSE
y_pred = model.predict(X)
rmse = mean_squared_error(y, y_pred, squared=False)

print(f"RMSE on filtered training data: {rmse:.4f}")

/tmp/ipykernel_36728/4143592312.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
/tmp/ipykernel_36728/4143592312.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


Number of features: 515
RMSE on filtered training data: 7.8118
